## Data Exploration

This notebook includes the code used for data-exploration. It has a cell to pull pip depencenies and some comments on my thinking process.


_IMPORTANT_: Please ensure that the variable `SAMPLES_DIR` points to the directory that contains all the JSON samples before running this notebook. Please make sure that this is not the same directory the ETL pipeline will point to as duplicate files may be created during this exploration.

### A summary of the process

![If this text is showing, please refer to root_dir/images/0_inspection.jpg](./images/0_inspection.jpg)


In [2]:
!pip install genson json-repair pandas

In [1]:

from genson import SchemaBuilder
import glob
import json
from json_repair import repair_json
import os
import pandas as pd


### Configuration

Update the samples directory to the folder where all your JSON samples are stored.

In [2]:
SAMPLES_DIR = 'data/mixed/'

### Global variables

- Track the max no of programs and tasks in a JSON payload to understand the nesting depth required for normalization.

In [3]:
max_advocacy_programs = 0
max_tasks_completed = 0


### Discussion


`Genson` was used to infer the schema for the JSON samples. The benefits are:
- To understand the structure of the JSON files.
- To understand what sort of data has been provided in terms of data types.
- To understand if there are missing fields across samples - fields present in all docs will be flagged as required
- To create a rough schema which we could update for validation purposes down the line.


In [4]:
def infer_schema_from_samples(samples_dir, on_sample_read=None):
    builder = SchemaBuilder()
    for filename in os.listdir(samples_dir):
        if filename.endswith('.json'):
            with open(os.path.join(samples_dir, filename), 'r', encoding='utf-8') as f:
                try:
                    data = json.load(f)
                    builder.add_object(data)
                    if on_sample_read is not None:
                        on_sample_read(data)
                except Exception as e:
                    pass

    return builder.to_schema()


In [5]:
def analyse_nesting(json_payload):
    global max_advocacy_programs, max_tasks_completed

    programs = json_payload.get('advocacy_programs', [])
    if isinstance(programs, list) and len(programs) > 0:
        if len(programs) > max_advocacy_programs:
            max_advocacy_programs = len(programs)

        for program in programs:
            tasks = program.get('tasks_completed', [])
            if isinstance(tasks, list) and len(tasks) > max_tasks_completed:
                max_tasks_completed = len(tasks)

In [6]:
def perform_structural_fix(file_path):
    broken_json = ""
    fixed_file_path =  file_path.replace('.json', '_fixed.json')
    with open(file_path, 'r', encoding='utf-8') as f:
        broken_json = f.read()

    fixed_json = repair_json(broken_json)
    if fixed_json:
        with open(fixed_file_path, 'w', encoding='utf-8') as f:
            f.write(fixed_json)
        return True, fixed_file_path
    return False, ""

In [7]:
def load_samples_df(samples_dir):
    structural_issue_count = 0
    needs_intervention = 0

    batch_files = glob.glob(samples_dir + '/*.json')
    batch_df = []
    for filename in batch_files:
        structural_fix_failed = False
        file_to_read = filename
        while not structural_fix_failed:
            try:
                df = pd.read_json(file_to_read)
                batch_df.append(df)
                break
            except:
                structural_issue_count += 1
                status, fixed_file_path = perform_structural_fix(filename)
                file_to_read = fixed_file_path
                if not status:
                    needs_intervention += 1
                    structural_fix_failed = True


    return pd.concat(batch_df), {'issue': structural_issue_count, 'needs_intervention': needs_intervention}

In [8]:
schema = infer_schema_from_samples(SAMPLES_DIR, on_sample_read=analyse_nesting)
samples_df, structure = load_samples_df(SAMPLES_DIR)

### Findings


#### Structure:
The following can be observed of the document structure:
- Records hold information of a single Program and a single activity within that program.
- Some JSON files have structural issues requiring correction before they can be processed.

In [9]:
print("\nFiles with Structural Issues: ", structure["issue"])
print("\nFiles needing manual intervention: ", structure["needs_intervention"])
print("\nNesting Depth Analysis: ", f"\n\tMax Advocacy Programs: {max_advocacy_programs}, \n\tMax Tasks Completed: {max_tasks_completed}")


Files with Structural Issues:  222

Files needing manual intervention:  0

Nesting Depth Analysis:  
	Max Advocacy Programs: 1, 
	Max Tasks Completed: 1


#### Schema:

The generated schema showed us that:

- Our data is roughly consistent structurally across samples: There are no columns that have been missed out in some samples or added to a select few. Additionally, there are no columns that have been mispelt in some samples.
- `name`, `email`, `tiktok_handle`, `joined_at`, `post_url` & `program_id`: are always strings indicating that the errors contained within them would be related to content accuracy and not type.
- `advocacy_programs` & `tasks_completed`: are always arrays indicating that the errors contained within them would be related to content accuracy and not type.
- `user_id`, `instagram_handle`, `task_id`, & `comments`: contain null values

similarly, we can see the numeric fields and where nulls are present.

In [10]:
display(schema)

{'$schema': 'http://json-schema.org/schema#',
 'type': 'object',
 'properties': {'user_id': {'type': ['null', 'string']},
  'name': {'type': 'string'},
  'email': {'type': 'string'},
  'instagram_handle': {'type': ['null', 'string']},
  'tiktok_handle': {'type': 'string'},
  'joined_at': {'type': 'string'},
  'advocacy_programs': {'type': 'array',
   'items': {'type': 'object',
    'properties': {'program_id': {'type': 'string'},
     'brand': {'type': ['integer', 'string']},
     'tasks_completed': {'type': 'array',
      'items': {'type': 'object',
       'properties': {'task_id': {'type': ['null', 'string']},
        'platform': {'type': ['integer', 'string']},
        'post_url': {'type': 'string'},
        'likes': {'type': ['integer', 'string']},
        'comments': {'type': ['integer', 'null']},
        'shares': {'type': 'integer'},
        'reach': {'type': 'integer'}},
       'required': ['comments',
        'likes',
        'platform',
        'post_url',
        'reach',
  

#### Data description:

Additionally, we used pandas to get a description of the data to understand the distribution of values in each column.

- Some columns had null values
- Some columns had unexpected values such as "???" in name, giving an idea of what sort of validation will be needed
- Some columns  had repeated values

In [13]:
display(samples_df.describe(include='all'))

,user_id,name,email,instagram_handle,tiktok_handle,joined_at,advocacy_programs
count,5489,10222,10222,5220,10222,10222,10222
unique,5465,5217,5717,5202,5206,5334,9992
top,ff389b31-4090-451a-aeee-1b87283034dd,???,invalid-email,@Cathy36,#error_handle,not-a-date,"{'program_id': '', 'brand': 'Luettgen, Gutmann..."
freq,2,4985,4463,2,4997,4880,2


Null checks

In [14]:
print(samples_df.isnull().sum())

user_id              4733
name                    0
email                   0
instagram_handle     5002
tiktok_handle           0
joined_at               0
advocacy_programs       0
dtype: int64


In [38]:
"""
Rough normalisation to allow us inspect nested fields - see etl pipeline for production ready version
"""
def normalise_data(df):
    df = df.melt(
        id_vars=['user_id', 'name', 'email', 'instagram_handle', 'tiktok_handle', 'joined_at'],
        value_vars=['advocacy_programs']).drop('variable', axis=1)
    df = df.reset_index(drop=True)
    normalised_advocacy_programs = pd.json_normalize(df['value']).fillna("").reset_index(drop=True)
    df = df.join(normalised_advocacy_programs)
    df = df.explode('tasks_completed')
    is_duplicated = df.index.duplicated(keep='first')
    df.loc[is_duplicated, 'total_sales_attributed'] = 0
    df = df.reset_index(drop=True)
    normalised_tasks_completed = pd.json_normalize(df['tasks_completed'])
    df_unique_tasks = normalised_tasks_completed.drop_duplicates(subset=['task_id', 'platform', 'post_url', 'likes', 'comments', 'shares', 'reach'])
    df = df.join(df_unique_tasks, how='right').reset_index(drop=True)
    final_columns = ['user_id', 'name', 'email', 'instagram_handle', 'tiktok_handle', 'joined_at',          'program_id',
                     'brand', 'total_sales_attributed', 'task_id', 'platform', 'post_url', 'likes', 'comments',
                     'shares', 'reach']
    return df[final_columns]


normalised_df = normalise_data(samples_df)

normalised_df[['user_id', 'program_id', 'task_id', 'email', 'instagram_handle', 'tiktok_handle', 'post_url', 'platform']] = (
    normalised_df[['user_id', 'program_id', 'task_id', 'email', 'instagram_handle', 'tiktok_handle', 'post_url', 'platform']]
    .astype(str)
)
normalised_df['name'] = normalised_df['name'].astype(str)
normalised_df['brand'] = normalised_df['brand'].astype(str)
normalised_df['platform'] = normalised_df['platform'].astype(str)
normalised_df.loc[normalised_df['platform'] == 'Tiktok', 'platform'] = 'TikTok'
normalised_df['joined_at'] = normalised_df['joined_at'].astype(str)
normalised_df['likes'] = pd.to_numeric(normalised_df['likes'], errors='coerce').astype('Int64')
normalised_df['comments'] = pd.to_numeric(normalised_df['comments'], errors='coerce').astype('Int64')
normalised_df['shares'] = pd.to_numeric(normalised_df['shares'], errors='coerce').astype('Int64')
normalised_df['reach'] = pd.to_numeric(normalised_df['reach'], errors='coerce').astype('Int64')
# this was set to the observed max value so we can observe the real mean value
normalised_df['total_sales_attributed'] = pd.to_numeric(normalised_df['total_sales_attributed'], errors='coerce').astype('Float64').fillna(4998.365222)

In [39]:
import pandas as pd

# Assuming df is your DataFrame
print("Max values per column:")
print(normalised_df.max(numeric_only=True))

print("\nMin values per column:")
print(normalised_df.min(numeric_only=True))

Max values per column:
total_sales_attributed    4998.365222
likes                           500.0
comments                         50.0
shares                          100.0
reach                          9999.0
dtype: Float64

Min values per column:
total_sales_attributed    101.77583
likes                          10.0
comments                        0.0
shares                          0.0
reach                       -1000.0
dtype: Float64


More statistical information observed after normalisation:

In [41]:
display(normalised_df.describe(include='all'))

,user_id,name,email,instagram_handle,tiktok_handle,joined_at,program_id,brand,total_sales_attributed,task_id,platform,post_url,likes,comments,shares,reach
count,9989,9989,9989,9989,9989,9989,9989,9989,9989.0,9989,9989,9989,5968.0,6424.0,9989.0,9989.0
unique,5462,5215,5713,5202,5204,5334,5499,4848,<NA>,5477,4,5943,<NA>,<NA>,<NA>,<NA>
top,nan,???,invalid-email,nan,#error_handle,not-a-date,,12345,<NA>,None,TikTok,broken_link,<NA>,<NA>,<NA>,<NA>
freq,4528,4764,4273,4785,4779,4656,4491,4523,<NA>,4513,3767,4046,<NA>,<NA>,<NA>,<NA>
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3530.830299,NaN,NaN,NaN,253.440851,24.868306,50.149665,5050.643107
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1623.080917,NaN,NaN,NaN,142.857539,14.637392,29.322314,2870.871179
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.77583,NaN,NaN,NaN,10.0,0.0,0.0,-1000.0
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2171.347406,NaN,NaN,NaN,128.0,12.0,25.0,2658.0
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4189.189813,NaN,NaN,NaN,253.0,25.0,50.0,5103.0
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4998.365222,NaN,NaN,NaN,378.0,37.0,76.0,7476.0
